# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2021'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400112','0400129']

#north_pumas = ['0400112','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC'

ADJHSG - adjustment factor for housing dollar amounts (6 decimal places)
1000000 = 1.000000

FHINCP - income flag - 1: yes | https://api.census.gov/data/2021/acs/acs1/pums/variables/FHINCP.json

HINCP - HHI in past 12 months (not -60000:n/a, 0:no income, -59999 loss of 59k+)
-1 to -59998 loss, 1+
https://api.census.gov/data/2021/acs/acs1/pums/variables/HINCP.json

SMOCP - selected owner costs (monthly) (not '00000': none / '-1')
https://api.census.gov/data/2021/acs/acs1/pums/variables/SMOCP.json

In [4]:
#Income to afford median household sales price by PUMA
inc_needed = {'0400112':178306,'0400113':132721,'0400114':89658,'0400115':83532,\
              '0400116':88144,'0400117':116721,'0400118':83279,'0400119':79856,\
              '0400120':110559,'0400121':85225,'0400122':61405,'0400123':68829,\
              '0400125':73694,'0400128':90883,'0400129':127279}

In [5]:
#AMI bands in 2021 by household size
AMI_30pct = {'1':16600,'2':19000,'3':21960,'4':26500,'5':31040,'6':35580,\
            '7':40120,'8':44660}
AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71100,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [6]:
#Housing costs affordable to different AMI bands based on household size
unit_afford = {'1':[0,461,767,1217,1535,1000000000],\
              '2':[0,527,877,1392,1535,1000000000],\
              '3':[0,609,985,1565,1973,1000000000],\
              '4':[0,735,1096,1738,2192,1000000000],\
              '5':[0,861,1185,1878,2370,1000000000],\
              '6':[0,987,1272,2017,2545,1000000000],\
              '7':[0,1113,1360,2156,2720,1000000000],\
              '8':[0,1239,1447,2295,2894,1000000000]}

inc_lbls = ['u30_ami','30_50_ami','50_80_ami','80_100_ami','o100_ami']

In [7]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [8]:
df = get.get_puma(sample,y1,data_cols)

In [9]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [10]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)


In [11]:
df['inc_needed'] = df['GEO_ID'].map(inc_needed)

In [12]:
dff = df[~(df.HHSz.isna())&(df.HINCP!=-60000)&(df.HINCP!=0)].copy()
dff['HINCP'] = dff.ADJINC * dff.HINCP

dff['aff_cost'] = np.where(dff.HINCP>=1,(dff.HINCP*0.333)/12,0)

In [13]:
dff['can_buy'] = np.where(dff.HINCP>=dff.inc_needed,'can buy','cannot afford')

In [14]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),'u30_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),'30_50_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),\
                   '50_80_ami',np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),\
                   '80_100_ami','o100_ami'))))

In [15]:
inc_needed = {'0400112':178306,'0400113':132721,'0400114':89658,'0400115':83532,\
              '0400116':88144,'0400117':116721,'0400118':83279,'0400119':79856,\
              '0400120':110559,'0400121':85225,'0400122':61405,'0400123':68829,\
              '0400125':73694,'0400128':90883,'0400129':127279}

In [16]:
#make a column for each PUMA that assesses the number of households 
own_cols=[]
for k in inc_needed.keys():
    dff[f'aff_{k}_can']=np.where(dff.HINCP>=inc_needed[k],1,0)
    dff[f'aff_{k}_cant']=np.where(dff.HINCP<inc_needed[k],1,0)
    dff[f'aff_{k}_can_wt']=dff[f'aff_{k}_can']*dff.WGTP
    dff[f'aff_{k}_cant_wt']=dff[f'aff_{k}_cant']*dff.WGTP
    own_cols.append(f'aff_{k}_can')
    own_cols.append(f'aff_{k}_cant')
    own_cols.append(f'aff_{k}_can_wt')
    own_cols.append(f'aff_{k}_cant_wt')

In [17]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,...,aff_0400125_can_wt,aff_0400125_cant_wt,aff_0400128_can,aff_0400128_cant,aff_0400128_can_wt,aff_0400128_cant_wt,aff_0400129_can,aff_0400129_cant,aff_0400129_can_wt,aff_0400129_cant_wt
3601,0400122,297649.1920,2.0,72.0,1.029928,69.0,19.0,68.0,74.0,21.0,...,72.0,0.0,1,0,72.0,0.0,1,0,72.0,0.0
3602,0400112,76214.6720,2.0,55.0,1.029928,56.0,96.0,91.0,53.0,16.0,...,55.0,0.0,0,1,0.0,55.0,0,1,0.0,55.0
3606,0400119,40476.1704,3.0,50.0,1.029928,51.0,87.0,16.0,92.0,87.0,...,0.0,50.0,0,1,0.0,50.0,0,1,0.0,50.0


### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened

In [18]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [19]:
drop_cols = ['HINCP','NP','ADJINC','HHSz','inc_needed']

In [20]:
table = dff.copy().drop(columns=drop_cols)

In [21]:
table_2 = table.copy().drop(columns=own_cols).groupby(['GEO_ID','AMI_range','can_buy']).sum().reset_index()
table_2 = make_est(table_2)
table_2 = table_2.drop(columns=repwts)
table_2 = pd.pivot_table(table_2,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','can_buy'],aggfunc=np.sum).reset_index()

In [22]:
table_3 = table.copy().drop(columns=['AMI_range','can_buy']).groupby(['GEO_ID']).sum().reset_index()

In [24]:
table_3 = make_est(table_3)
table_3 = table_3.drop(columns=repwts)

In [28]:
with pd.ExcelWriter(f'output/pums_own_gap_marketdata.xlsx') as writer:
    table_2.to_excel(writer,sheet_name='can_cannot_withinpuma.xlsx')
    table_3.to_excel(writer,sheet_name='can_cannot_atpuma.xlsx')